In [2]:
# Import modules
import argparse
import logging
import cell2cell as c2c
import pandas as pd
import scprep
import os
import numpy as np
import pyreadr as py
from pathlib import Path
import pickle

In [3]:
def parse_path(p):
    if os.path.islink(p):
        return os.readlink(p)
    else:
        return p

In [25]:
project_dir = "/Users/joankant/Desktop/gaitigroup/Users/Joan/h4h-cell-cell-interactions"

In [38]:
args = dict()
input_dir = f"{project_dir}/output/CellClass_L4_min3_types_rerun/100_preprocessing/mtx/6467_solid_core"
meta =f"{project_dir}/output/CellClass_L4_min3_types_rerun/000_data/seurat_annot_adapted__metadata.csv"
output_dir = f"{project_dir}/output/test_output"
interactions = f"{project_dir}/001_data_local/interactions_db_v2/liana_db.csv"
annot = "CellClass_L4"
nperm = 10
sample_id="6467_solid_core"

In [43]:
# Setup logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(message)s')

# Create output directories if they don't exist
logging.info("Check if output directory exists...")

path = Path(output_dir)
if not path.exists():
    logging.info("Output directory does not exist. Creating it...")
    path.mkdir(parents=True, exist_ok=True)
else:
    logging.info("Output directory already exists.")

# if not os.path.exists(output_dir):
#     logging.info("Output directory does not exist. Creating it...")
#     os.makedirs(output_dir)

# if (((run_grid is not None) & (run is not None) & (sampling_path is not None) & (input_dir is not None))):
#     # Load sampling grid
#     logging.info("Load sampling grid...")
#     run_grid = pd.read_csv(f"{run_grid}")

#     # Extract info for current sample
#     sample_id = os.path.splitext(run_grid.sample_id[run])[0]
#     iteration = run_grid.id[run]

#     logging.info("Load data...")
#     logging.info(f"{parse_path(input_dir)}/{sample_id}")
#     rnaseq = scprep.io.load_10X(
#         f"{parse_path(input_dir)}/{sample_id}").T
# else:
if (os.path.exists(input_dir)):
    rnaseq_df = scprep.io.load_10X(parse_path(input_dir)).T
else:
    raise Exception("Not a valid path for the input file")
# Set genes as index
rnaseq_df = rnaseq_df.rename_axis("index", axis=1)
# rnaseq = rnaseq.to_df()


2023-12-19 12:25:04,628 Check if output directory exists...
2023-12-19 12:25:04,635 Output directory already exists.


In [35]:
rnaseq

index,6467_solid_core_AAACAGCCAGCAACCT-1,6467_solid_core_AAACAGCCATGGAGGC-1,6467_solid_core_AAACAGCCATGTGGGA-1,6467_solid_core_AAACATGCACGAACAG-1,6467_solid_core_AAACCAACAATTGAGA-1,6467_solid_core_AAACCAACAGCCTAAC-1,6467_solid_core_AAACCGAAGACTTATG-1,6467_solid_core_AAACCGAAGGCAATAG-1,6467_solid_core_AAACCGAAGGTGAAAT-1,6467_solid_core_AAACCGAAGTTATCCT-1,...,6467_solid_core_TTTGTGAAGGCCCAGT-1,6467_solid_core_TTTGTGGCAACCCTAA-1,6467_solid_core_TTTGTGGCAACTGGCT-1,6467_solid_core_TTTGTGGCACCTATAG-1,6467_solid_core_TTTGTGTTCATGTTTC-1,6467_solid_core_TTTGTGTTCGGTCAGC-1,6467_solid_core_TTTGTGTTCGGTCATG-1,6467_solid_core_TTTGTTGGTGGTGAGA-1,6467_solid_core_TTTGTTGGTTACTTGC-1,6467_solid_core_TTTGTTGGTTTCGCCA-1
NRG2,4.15104,0.0,0.000000,0.0,0.0,0.0,0.000000,2.600378,0.0,0.000000,...,0.0,0.0,0.000000,0.0,2.584367,0.000000,0.0,0.000000,0.0,0.0
EREG,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SEMA7A,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
NFASC,0.00000,0.0,0.000000,0.0,0.0,0.0,4.175964,2.600378,0.0,3.113754,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
TGFA,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLC22A17,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,3.255696,0.0,0.000000,...,0.0,0.0,0.000000,0.0,2.584367,4.585859,0.0,4.221184,0.0,0.0
TLR8,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SV2A,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,3.136643,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SV2B,0.00000,0.0,4.538951,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [42]:

    print(rnaseq.head())
    print(rnaseq.index)
    # logging.info("Select cells for iteration...")
    # if (sampling_path is not None):
    #     # Read sampling grid for current sample
    #     sampling_grid = py.read_r(
    #         f"{sampling_path}/{sample_id}__grid.rds")[None]
    #     # Select the cell ids for this iteration
    #     cell_ids = sampling_grid.loc[sampling_grid.run == iteration,
    #                                  sampling_grid.columns != "run"].to_numpy().flatten().tolist()
    #     # Select the cells
    # rnaseq = rnaseq.loc[:, rnaseq.columns.isin(cell_ids)]

    logging.info("Load database with CCIs...")
    lr_pairs = pd.read_csv(interactions)
    lr_pairs = lr_pairs.astype(str)

    # Metadata for the single cells
    logging.info("Load metadata...")
    meta_df = pd.read_csv(meta, index_col=0)
    meta_df.index.name = "index"
    meta_df = meta_df.loc[meta_df.index.isin(rnaseq.columns)]

2023-12-19 12:24:38,482 Load database with CCIs...


index   6467_solid_core_AAACAGCCAGCAACCT-1  \
NRG2                               4.15104   
EREG                               0.00000   
SEMA7A                             0.00000   
NFASC                              0.00000   
TGFA                               0.00000   

index   6467_solid_core_AAACAGCCATGGAGGC-1  \
NRG2                                   0.0   
EREG                                   0.0   
SEMA7A                                 0.0   
NFASC                                  0.0   
TGFA                                   0.0   

index   6467_solid_core_AAACAGCCATGTGGGA-1  \
NRG2                                   0.0   
EREG                                   0.0   
SEMA7A                                 0.0   
NFASC                                  0.0   
TGFA                                   0.0   

index   6467_solid_core_AAACATGCACGAACAG-1  \
NRG2                                   0.0   
EREG                                   0.0   
SEMA7A                         

2023-12-19 12:24:39,934 Load metadata...


In [49]:
rnaseq

index,6467_solid_core_AAACAGCCAGCAACCT-1,6467_solid_core_AAACAGCCATGGAGGC-1,6467_solid_core_AAACAGCCATGTGGGA-1,6467_solid_core_AAACATGCACGAACAG-1,6467_solid_core_AAACCAACAATTGAGA-1,6467_solid_core_AAACCAACAGCCTAAC-1,6467_solid_core_AAACCGAAGACTTATG-1,6467_solid_core_AAACCGAAGGCAATAG-1,6467_solid_core_AAACCGAAGGTGAAAT-1,6467_solid_core_AAACCGAAGTTATCCT-1,...,6467_solid_core_TTTGTGAAGGCCCAGT-1,6467_solid_core_TTTGTGGCAACCCTAA-1,6467_solid_core_TTTGTGGCAACTGGCT-1,6467_solid_core_TTTGTGGCACCTATAG-1,6467_solid_core_TTTGTGTTCATGTTTC-1,6467_solid_core_TTTGTGTTCGGTCAGC-1,6467_solid_core_TTTGTGTTCGGTCATG-1,6467_solid_core_TTTGTTGGTGGTGAGA-1,6467_solid_core_TTTGTTGGTTACTTGC-1,6467_solid_core_TTTGTTGGTTTCGCCA-1
NRG2,4.15104,0.0,0.000000,0.0,0.0,0.0,0.000000,2.600378,0.0,0.000000,...,0.0,0.0,0.000000,0.0,2.584367,0.000000,0.0,0.000000,0.0,0.0
EREG,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SEMA7A,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
NFASC,0.00000,0.0,0.000000,0.0,0.0,0.0,4.175964,2.600378,0.0,3.113754,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
TGFA,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SLC22A17,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,3.255696,0.0,0.000000,...,0.0,0.0,0.000000,0.0,2.584367,4.585859,0.0,4.221184,0.0,0.0
TLR8,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SV2A,0.00000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,3.136643,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0
SV2B,0.00000,0.0,4.538951,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0


In [50]:


    # Cell-cell Interactions and Communication Analysis
    # The pipeline integrates the RNA-seq and PPI datasets by using the analysis setups.
    # It generates an interaction space containing an instance for each sample/cell type, containing the values assigned to each protein in the PPI list given the setups for computing the CCI and CCC scores.

    logging.info("Infer CCIs")
    interactions = c2c.analysis.SingleCellInteractions(rnaseq_data=rnaseq,
                                                       ppi_data=lr_pairs,
                                                       metadata=meta_df,
                                                       interaction_columns=(
                                                           'source_genesymbol', 'target_genesymbol'),
                                                       communication_score='expression_gmean',
                                                       cci_score='bray_curtis',
                                                       cci_type='directed',
                                                       aggregation_method='average',
                                                       barcode_col='index',
                                                       celltype_col=annot,
                                                       complex_sep='_',
                                                       verbose=True)

    # **Compute communication scores for each PPI or LR pair**
    interactions.compute_pairwise_communication_scores()
    logging.info("Perform permutation analysis...")
    interactions.permute_cell_labels(evaluation='communication',
                                                permutations=nperm,
                                                fdr_correction=True,
                                                verbose=True)
    logging.info("Save results...")
    interactions.ccc_permutation_pvalues.to_csv(
        f"{output_dir}/cell2cell__{sample_id}.csv", sep="\t")

    logging.info("Save interactions object as pickle...")
    with open(f"{output_dir}/cell2cell__{sample_id}.pickle", "wb") as handle:
        pickle.dump(interactions, handle, protocol=pickle.HIGHEST_PROTOCOL)

    logging.info("COMPLETED")


2023-12-19 12:26:09,049 Infer CCIs


Removing bidirectionality of PPI network


/Users/joankant/miniforge3/envs/cell2cell/lib/python3.10/site-packages/cell2cell/preprocessing/rnaseq.py:250: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df.reset_index(inplace=True)


Creating Interaction Space
Generating objects according to RNAseq datasets provided
New cell instance created for Malignant
New cell instance created for Microglia
New cell instance created for Oligodendrocyte


2023-12-19 12:26:11,722 Perform permutation analysis...


Computing pairwise communication
Computing communication score between Malignant and Microglia
Computing communication score between Malignant and Malignant
Computing communication score between Microglia and Oligodendrocyte
Computing communication score between Oligodendrocyte and Oligodendrocyte
Computing communication score between Microglia and Microglia
Computing communication score between Oligodendrocyte and Malignant
Computing communication score between Microglia and Malignant
Computing communication score between Oligodendrocyte and Microglia
Computing communication score between Malignant and Oligodendrocyte


100%|██████████| 10/10 [00:23<00:00,  2.36s/it]
2023-12-19 12:26:39,547 Save results...
2023-12-19 12:26:39,904 Save interactions object as pickle...
2023-12-19 12:26:41,596 COMPLETED


In [53]:
pd.__version__

'1.3.5'